# Anomalib Models
In this notebook, we show how anomalib models could be initialized via Python API. As shown in [README.md](https://github.com/openvinotoolkit/anomalib#training), following models are supported in anomalib:

- [CFlow](anomalib/models/cflow)
- [DFM](anomalib/models/dfm)
- [DFKDE](anomalib/models/dfkde)
- [FastFlow](anomalib/models/fastflow)
- [PatchCore](anomalib/models/patchcore)
- [PADIM](anomalib/models/padim)
- [STFPM](anomalib/models/stfpm)
- [GANomaly](anomalib/models/ganomaly)


## Data Module
To train each model end-to-end, we do need to have a dataset. In this tutorial we will use MVTec AD DataModule. We assume that `datasets` directory is created in the `anomalib` root directory and `MVTec` dataset is located in `datasets` directory. Let's confirm this with the following:

In [ ]:
from pathlib import Path

Path.cwd()

In [ ]:
# Go to the main anomalib root dir
root = Path.cwd().parent.parent
root

In [ ]:
list((root / "datasets").iterdir())

Now that we checked we have `datasets` directory and `MVTec` is already located in `datasets`, we could create the datamodule.

In [ ]:
from anomalib.data.mvtec import MVTec

MVTec??

In [ ]:
datamodule = MVTec(
    root="../../datasets/MVTec/", 
    category="bottle",
    image_size=256,
    train_batch_size=32,
    test_batch_size=32,
    num_workers=8,
    task="segmentation",
)
datamodule.setup()
i, data = next(enumerate(datamodule.test_dataloader()))
data["image"].shape, data["mask"].shape

## Models
Now that we have created MVTec datamodule, we could create the models. We could start with `PatchCore` since it is the ranked #1 model on the MVTec AD category on papers with code. 

Each model on anomalib has the following structure:
```
        anomalib/models/<model_name>
        ├── README.md           # Readme file containing description and benchmarks.
        ├── __init__.py         # Model initialization.
        ├── config.yaml         # Stores the model configurations.  
        ├── torch_model.py      # Torch model implementing the basic forward-pass mechanism.
        ├── anomaly_map.py      # [Optional] module generating anomaly heatmaps.      
        ├── lightning_model.py  # Lightning module implementing training mechanism
        └── loss.py             # [Optional] module implementing loss computation. 
```



### Torch Model

In [ ]:
from anomalib.models.fastflow.torch_model import FastflowModel

FastflowModel??

In [ ]:
torch_model = FastflowModel(input_size=[256, 256], backbone="resnet18", flow_steps=8)

As mentioned, `torch_model` implements the basic forward-pass mechanism of the model for both `train` and `test` phases. In `train` phase, the model returns the training output such as feature maps. During the `test` phase, it returns the anomaly heatmap.

In [ ]:
torch_model.training = True
train_output = torch_model(data["image"])
hidden_variables, log_jacobian = train_output
hidden_variables[0].shape

In [ ]:
torch_model.training = False
anomaly_map = torch_model(data["image"])
anomaly_map.shape

As shown above, when the `training` is `False`, meaning the model is in val/test/inference stage, it produces the `anomaly_map`.

### Lightning Module
The main module for each anomalib models is the `LightningModule` that stores the torch_model as its attributes and sorts out the train-test mechanism. Let's see how the LightningModule of `Fastflow` model is instantiated.

In [ ]:
from anomalib.models.fastflow.lightning_model import Fastflow

Fastflow??

In [ ]:
model = Fastflow(
    input_size=[256, 256], 
    backbone="resnet18", 
    flow_steps=8,
)

In [ ]:
model.training = True
train_output = model(data["image"])
hidden_variables, log_jacobian = train_output
hidden_variables[0].shape

As can be seen above, the Lightning Module also returns the same output as `torch_model`. This is because it stores `torch_model` as its attribute and uses it in its `forward` method. Therefore, it is possible to call the forward-pass with `model(x)`. 

In [ ]:
model.model.training = False
anomaly_map = model(data["image"])
anomaly_map.shape

Similar to the `torch_model`, `lightning_module` also produces an `anomaly_map` when the `training` is set to `False`.